In [2]:
import requests
import numpy as np

# Download file
r = requests.get('https://user-web.icecube.wisc.edu/~grbweb_public/Summary_table.txt')
with open("Summary_table.txt", 'wb') as f:
    f.write(r.content)

# Read content
data = np.loadtxt("Summary_table.txt", dtype='str',unpack='True')

# Read headers
with open("Summary_table.txt",'r') as f:
    names= np.array([n.strip().replace(" ","_") for n in f.readlines()[1].replace("#","").replace("\n","").lstrip().split('    ') if n.strip()!=''])

print (names)

/tmp/ipykernel_1974/1317417187.py:10: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=50000`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an example on how to do this.  If you wish to ignore this warning, use `warnings.filterwarnings`.  This warning is expected to be removed in the future and is given only once per `loadtxt` call.
  data = np.loadtxt("Summary_table.txt", dtype='str',unpack='True')


['GRB_name' 'GRB_name_Fermi' 'T0' 'ra' 'decl' 'pos_error' 'T90'
 'T90_error' 'T90_start' 'fluence' 'fluence_error' 'redshift' "'T100'"
 'GBM_located' 'mjd_(T0)']


In [ ]:
# Trova indice della colonna
idx_time = np.where(names == 'T90')[0][0]
idx_err_time = np.where(names == 'T90_error')[0][0]
idx_fluence = np.where(names == 'fluence')[0][0]
idx_err_fluence = np.where(names == 'fluence_error')[0][0]

# Estrai colonne
time = data[idx_time].astype(float)
err_time = data[idx_err_time].astype(float)
fluence = data[idx_fluence].astype(float)
err_fluence = data[idx_err_fluence].astype(float)

# Costruisci dataset (una riga per evento, 4 colonne)
new_dataset = np.column_stack((time, err_time, fluence, err_fluence))

print("Shape:", new_dataset.shape)
for i in range(10):
    print(new_dataset[i])


Shape: (8721, 4)
Prima riga: [-999. -999. -999. -999.]
Prima riga: [1.0240e+01 9.2300e-01 2.5096e-06 3.0285e-08]
Prima riga: [-999. -999. -999. -999.]
Prima riga: [-999. -999. -999. -999.]
Prima riga: [2.2784e+01 1.0560e+00 3.8016e-05 6.7097e-08]
Prima riga: [1.1008e+01 1.1450e+00 1.1293e-06 2.2733e-08]
Prima riga: [2.68805e+02 1.71700e+00 1.77340e-05 6.30950e-08]
Prima riga: [6.2720e+00 1.4300e-01 4.5797e-05 1.7873e-08]
Prima riga: [6.0672e+01 2.1870e+00 2.0956e-06 3.0084e-08]
Prima riga: [-999. -999. -999. -999.]


In [ ]:
mask = np.all((new_dataset > -990) & (~np.isnan(new_dataset)), axis=1)
clean_dataset = new_dataset[mask]

for i in range(10):
    print(clean_dataset[i])

Prima riga: [1.0240e+01 9.2300e-01 2.5096e-06 3.0285e-08]
Prima riga: [2.2784e+01 1.0560e+00 3.8016e-05 6.7097e-08]
Prima riga: [1.1008e+01 1.1450e+00 1.1293e-06 2.2733e-08]
Prima riga: [2.68805e+02 1.71700e+00 1.77340e-05 6.30950e-08]
Prima riga: [6.2720e+00 1.4300e-01 4.5797e-05 1.7873e-08]
Prima riga: [6.0672e+01 2.1870e+00 2.0956e-06 3.0084e-08]
Prima riga: [2.0992e+01 1.1450e+00 5.9394e-06 5.2193e-08]
Prima riga: [9.1394e+01 7.2400e-01 3.4568e-05 3.5691e-08]
Prima riga: [1.6384e+01 1.2800e+00 2.8774e-06 2.0687e-08]
Prima riga: [4.4288e+01 1.8640e+00 3.6439e-06 2.4630e-08]
